In [7]:
from pyspark.sql import SparkSession


spark_session = SparkSession\
        .builder\
        .master("spark://10.0.6.155:7077") \
        .appName("test")\
        .config("spark.dynamicAllocation.enabled", False)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.executor.instances", 1)\
        .config("spark.executor.cores",2)\
        .getOrCreate()


spark_context = spark_session.sparkContext

In [8]:
from pyspark.sql.functions import col

In [9]:
import time

In [10]:
#%%time
#df = spark_session.read.csv("hdfs://hadoop-master:9000/user/root/input/repli_1000_MSD_FINAL2.csv",header=True,sep=",")
#df = spark_session.read.csv("hdfs://hadoop-master:9000/user/root/input/MSD_FINAL.csv",header=True,sep=",")

datasets = ["hdfs://hadoop-master:9000/user/root/input/MSD_FINAL.csv", "hdfs://hadoop-master:9000/user/root/input/repli_6000_MSD_FINAL.csv", "hdfs://hadoop-master:9000/user/root/input/repli_1000_MSD_FINAL2.csv", "hdfs://hadoop-master:9000/user/root/input/repli_150000_MSD_FINAL.csv"]
times = []
for dataset in datasets:
    start = time.time()
    df = spark_session.read.csv(dataset,header=True,sep=",")
    #print(df.count())
    print(df.rdd.getNumPartitions())

    cols = ['danceability','duration','energy','key', 'loudness', 'mode','tempo','year']
    df_2 = df.select(*(col(c).cast("float").alias(c) for c in cols))
    df_2 = df_2.filter(df.year > 0)
    df_3 = df_2.groupBy('year')
    df_avg = df_3.mean().orderBy('year', ascending=False)
    #df_avg.count()
#     df_avg.show(10)
    end = time.time()
    times.append(end-start)
    
times

1
4
13
74


[8.486099004745483, 5.054365873336792, 0.5134804248809814, 0.4343554973602295]

In [ ]:
df.rdd.getNumPartitions()

In [ ]:
df_2.head(5)

In [ ]:
df_avg.select('year').collect()

In [11]:
spark_context.stop()